In [2]:
import json

In [3]:
data=[]
with open('scierc_data.json', 'r') as file:
    for lines in file:
        # Process each line as needed
        # For example, if each line is a JSON object, you can parse it
        data.append(json.loads(lines))

In [4]:
new_data = []
for i in range(len(data)):
    cnt=0
    for j in range(len(data[i]['sentences'])):
        ner=[]
        for k in range(len(data[i]['ner'][j])):
            if data[i]['ner'][j][k]:
                
                start = data[i]['ner'][j][k][0]-cnt
                end= data[i]['ner'][j][k][1]-cnt
                word=' '.join(data[i]['sentences'][j][start:end+1])
                ner.append([start, end, word,data[i]['ner'][j][k][2]])
        relations=[]
        for k in range(len(data[i]['relations'][j])):
            if data[i]['relations'][j][k]:
                start = data[i]['relations'][j][k][0]-cnt
                end= data[i]['relations'][j][k][1]-cnt
                word1=' '.join(data[i]['sentences'][j][start:end+1])
                start = data[i]['relations'][j][k][2]-cnt
                end= data[i]['relations'][j][k][3]-cnt
                word2=' '.join(data[i]['sentences'][j][start:end+1])
                relations.append([word1, word2, data[i]['relations'][j][k][4]])
        cnt+=len(data[i]['sentences'][j])
            
        new_data.append({
            'sentence': data[i]['sentences'][j],
            'ner': ner,
            'relations': relations
        })

In [5]:
entity_labels=set()
entity_labels_true_count={}
other={}
for i in range(len(new_data)):
    for j in new_data[i]['ner']:
        if j[3] not in entity_labels_true_count:
            entity_labels_true_count[j[3]] = 0
        if j[3]=='OtherScientificTerm':
            if j[2] not in other:
                other[j[2]]=0
            other[j[2]]+=1
        entity_labels_true_count[j[3]] += 1
        entity_labels.add(j[3])

In [6]:
entity_labels

{'Generic', 'Material', 'Method', 'Metric', 'OtherScientificTerm', 'Task'}

In [ ]:
from openai import OpenAI
import json
import time
from tqdm import tqdm
import pandas as pd  # make sure df is defined with 'Content' column

# Set OpenAI API key
client = OpenAI(api_key = "")  # Replace with your actual API key

# Function: extract named_entities given allowed entity labels
def extract_named_entities_by_labels(paragraph, allowed_labels):
    allowed_labels_str = ", ".join(allowed_labels)

    system_prompt = (
        "You are a named entity recognition (NER) assistant. "
        "Your task is to extract named entities from a given paragraph, "
        "but only include entities whose type is one of the following: "
        f"{allowed_labels_str}. "
        "Return ONLY a JSON list of entity strings that match the allowed labels. "
        "Do not include the labels in the output. Keep it JSON parsable."
    )

    user_prompt = f"""
Paragraph:

{paragraph}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-4o-mini"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
        )

        reply = response.choices[0].message.content.strip()

        if reply.startswith("```json"):
            reply = reply[len("```json"):].strip()
        if reply.endswith("```"):
            reply = reply[:-len("```")].strip()

        return json.loads(reply)

    except Exception as e:
        print("Error with paragraph:", paragraph)
        print("Exception:", e)
        return []

    # Example: allowed labels
allowed_entity_labels = ['Generic', 'Material', 'Method', 'Metric', 'OtherScientificTerm', 'Task']

# Loop through DataFrame and extract entities
gpt_named_entities = []


for i in tqdm(range(len(new_data))):
    paragraph = ' '.join(new_data[i]['sentence'])
    entities = extract_named_entities_by_labels(paragraph, allowed_entity_labels)
    gpt_named_entities.append(entities)

100%|██████████| 551/551 [06:45<00:00,  1.36it/s]


In [10]:
with open("SCIERC_gpt_named_entities_output.json", "w") as f:
    json.dump(gpt_named_entities, f, indent=2)

In [9]:
true=[]
for i in range(len(new_data)):
    temp=[]
    for j in new_data[i]['ner']:
        temp.append(j[2])
    true.append(temp)

In [11]:
true[2]

['approach',
 'Multi-lingual Evaluation Task -LRB- MET -RRB-',
 'Japanese text',
 'task',
 'morphological analysis problem',
 'Japanese']

In [12]:
their=0
ours=0
common=0
for i in range(len(gpt_named_entities)):
    their+=len(set(true[i]))
    ours+=len(set(gpt_named_entities[i]))
    common+=len(set(true[i]).intersection(set(gpt_named_entities[i])))

In [13]:
their,ours,common

(1673, 584, 353)

In [14]:
recall_avg=[]
precision_avg=[]
common2=0
for i in range(len(gpt_named_entities)):
    true_set=set(true[i])
    pred_set=set(gpt_named_entities[i])
    
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if m in n or n in m:
                    count+= 1
        recall = count / len(true_set) if len(true_set) > 0 else 0
        precision = count / len(pred_set) if len(pred_set) > 0 else 0
        common2 += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [15]:
common2

545

In [16]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.29432533588613446, 0.30358223144067076)

In [17]:

for i in range(len(new_data)):
    for j in range(len(new_data[i]['ner'])):
        new_data[i]['ner'][j][2],new_data[i]['ner'][j][3] = new_data[i]['ner'][j][3], new_data[i]['ner'][j][2]

In [18]:
final={}
rel_count={}
for i in range(len(new_data)):
    mapp={}
    for j in new_data[i]['ner']:
        mapp[j[3]]=j[2]
    for j in new_data[i]['relations']:
        if j[2] not in rel_count:
            rel_count[j[2]] = 0
        rel_count[j[2]] += 1
        if j[2] not in final:
            final[j[2]] = set()
        final[j[2]].add((mapp[j[0]], mapp[j[1]]))

In [ ]:
from openai import OpenAI
import json
import re

# Initialize OpenAI client
client = OpenAI(api_key = "")  # Replace with your actual API key

def extract_relation_labels_with_gpt_entities(paragraph, entities, relation_labels):
    """
    Extract RDF triples using GPT-4o/mini with entity spans.

    Args:
        paragraph (str): Full input paragraph
        entities (list): Each entity as [start, end, entity_type, entity_text]
        relation_labels (list): Allowed relation labels

    Returns:
        dict: {"relation_triples": [[head, relation, tail], ...]}
    """
    # Convert entity structure into readable text
    entity_descs = [
        f"[{start}, {end}, {etype}, {text}]"
        for start, end, etype, text in entities
    ]
    entity_str = "\n".join(entity_descs)
    relation_str = ", ".join(relation_labels)

    system_prompt = (
        "You are an expert in information extraction. "
        "Given a paragraph, a list of named entities with character spans and types, and a list of allowed relation labels, "
        "extract RDF relation triples in the format [head, relation, tail].\n\n"
        "- Head and tail must be from the provided entity list.\n"
        "- The relation must be from the relation_labels list.\n"
        "- The output must be ONLY a JSON object like:\n"
        '{ "relation_triples": [ ["Entity1", "Relation", "Entity2"], ... ] }\n'
        "- Do NOT include any explanation or extra text."
    )

    user_prompt = f"""
Paragraph:
\"\"\"
{paragraph}
\"\"\"

Entities (format: [start, end, type, entity]):
{entity_str}

Relation labels:
{relation_str}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-4o-mini"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
        )

        reply = response.choices[0].message.content.strip()

        # Extract JSON block
        json_match = re.search(r'\{.*\}', reply, re.DOTALL)
        if json_match:
            json_text = json_match.group(0).strip()
            return json.loads(json_text)
        else:
            print("No JSON found.")
            print("Reply:", reply)
            return {"relation_triples": []}

    except Exception as e:
        print("Exception:", e)
        return {"relation_triples": []}
gpt_relation_triples = []
relation_labels = ['USED-FOR','CONJUNCTION', 'EVALUATE-FOR', 'FEATURE-OF', 'PART-OF','COMPARE']

for i in tqdm(range(len(new_data))):
    paragraph = ' '.join(new_data[i]['sentence'])
    entities = new_data[i]['ner']
    output= extract_relation_labels_with_gpt_entities(paragraph, entities, relation_labels)
    gpt_relation_triples.append(output['relation_triples'])

100%|██████████| 551/551 [10:03<00:00,  1.10s/it]


In [23]:
with open("SCIERC_gpt_relation_triples_output.json", "w") as f:
    json.dump(gpt_relation_triples, f, indent=2)

In [27]:
gpt_relation_triples[2]

[['approach', 'USED-FOR', 'Multi-lingual Evaluation Task -LRB- MET -RRB-'],
 ['Multi-lingual Evaluation Task -LRB- MET -RRB-',
  'EVALUATE-FOR',
  'Japanese text'],
 ['task', 'FEATURE-OF', 'morphological analysis problem'],
 ['morphological analysis problem', 'PART-OF', 'Japanese']]

In [26]:
relations_true=[]
for i in range(len(new_data)):
    
    relations_true.append(new_data[i]['relations'])

In [43]:
relations_true[2]

[['approach', 'Multi-lingual Evaluation Task -LRB- MET -RRB-', 'USED-FOR'],
 ['Multi-lingual Evaluation Task -LRB- MET -RRB-',
  'Japanese text',
  'USED-FOR'],
 ['morphological analysis problem', 'task', 'USED-FOR'],
 ['Japanese', 'morphological analysis problem', 'USED-FOR']]

In [30]:
recall_avg=[]
precision_avg=[]
common=0
ours=0
relations_all_filtered=[]

for i in range(len(relations_true)):
    true_set=relations_true[i]
    pred_set=gpt_relation_triples[i]
    d={}
    temp=[]
    for item in pred_set:
        if (item[0],item[2]) not in d and (item[2],item[0]) not in d:
            d[(item[0],item[2])]=item[1]
            temp.append([item[0],item[1],item[2]])
    relations_all_filtered.append(temp)
    
    ours+=len(d)
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if m[0]==n[0] and m[1]==n[2] and (m[2]==n[1]):
                    count+= 1
        recall = count/ len(true_set) if len(true_set) > 0 else 0
        precision = count / len(pred_set) if len(pred_set) > 0 else 0
        common += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [31]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.42119097377718073, 0.4164419669864318)